# Cvičení 21.2.2022

#### Naimplementujte Vigenerovu šifru

- Pracujte s přípustnou abecedou obsahující jen znaky anglické abecedy bez mezery. 
- Implementujte jak šifrování, tak dešifrování. Pro kontrolu použijte klíč K=ZIMA a zašifrujte tento otevřený text: 

Fungování šifry, jde o to, že používáme stále stejný posun jako v Ceaserove šifře. Akorát s tím rozdílem, že se každý znak
šifruje dle klíče, který se opakuje.

In [17]:
import numpy as np
import pandas as pd

In [11]:
def generate_alphabet(start, end):
    alphabet = {}
    for i in range(start, end+1):
        c = chr(i)
        alphabet[c] = i

    return alphabet

In [21]:
alphabet = generate_alphabet(65, 90)

In [44]:
def generate_vigener_matrix(alphabet):
    l = len(alphabet)
    alphabet_c = list(alphabet.keys())
    matrix = np.zeros(shape=(l, l), dtype=str)

    for r_i in range(0, l):
        for c_i in range(0, l):
            matrix[r_i, c_i] = alphabet_c[(c_i + r_i) % l]  


    return pd.DataFrame(matrix, index=alphabet, columns=alphabet)

In [45]:
vigener_matrix = generate_vigener_matrix(alphabet)

In [100]:
current_index = list(vigener_matrix.loc[:, 'C'].values).index('C')

In [101]:
vigener_matrix.iloc[current_index, 0]

'A'

In [102]:
class VigenerCipher:
    def __init__(self, alphabet):
        self.alphabet = alphabet
        self.vigener_matrix = generate_vigener_matrix(alphabet)

    def encode(self, oa_text, key):
        sa_text = ""
        size_key = len(key)
        chars_key = list(key)

        for index_oa, char_oa in enumerate(oa_text):
            current_key_index = index_oa % size_key  
            current_key_char = chars_key[current_key_index]

            sa_text += self.vigener_matrix.loc[char_oa, current_key_char]


        return sa_text
    
    def decode(self, sa_text, key):
        oa_text = ""
        size_key = len(key)
        chars_key = list(key)

        for index_sa, char_sa in enumerate(sa_text):
            current_key_index = index_sa % size_key  
            current_key_char = chars_key[current_key_index]

            current_index = list(vigener_matrix.loc[:, current_key_char].values).index(char_sa)
            current_oa_char = vigener_matrix.iloc[current_index, 0]

            oa_text += current_oa_char


        return oa_text

In [103]:
v_c = VigenerCipher(alphabet)

In [104]:
oa_vigener_text = "SLUNECNEATEPLEPOCASICTVRTKEMPROZATIMKONCIVNASLEDUJICICHDVOUDNECHSEZATAHNEVETSINUUZEMIZASAHNEDESTNAHORACHANASEVEROVYCHODEVEVSECHPOLOHACHSNIHOVIKENDUBUDEPOLOJASNOTEPLOTYVNOCIKLESNOUPODNULUPRESDENBUDEMAXIMALNESESTSTUPNU"

In [105]:
sa_vigener_text = "RTGNDKZEZBQPKMBOBIEIBBHRSSQMOZAZZBUMJWZCHDZARTQDTRUCHKTDUWGDMMOHRMLASITNDDQTRQZUTHQMHHMSZPZECMETMITOQIOHZVMSDDQRNDKCGWPEUMHSDKTPNTAHZKTSMQTOUQWEMLGBTLQPNTAJZAZOSMBLNBKVMWOIJTQSMWGPNLZUKCBRDAPEMJGDDUMXHUMLMMEERBETTXZU"

In [106]:
specified_key = "ZIMA"

#### Is ok encoding?

In [112]:
sa_text = v_c.encode(oa_vigener_text, specified_key)

In [113]:
sa_text

'RTGNDKZEZBQPKMBOBIEIBBHRSSQMOZAZZBUMJWZCHDZARTQDTRUCHKTDUWGDMMOHRMLASITNDDQTRQZUTHQMHHMSZPZECMETMITOQIOHZVMSDDQRNDKCGWPEUMHSDKTPNTAHZKTSMQTOUQWEMLGBTLQPNTAJZAZOSMBLNBKVMWOIJTQSMWGPNLZUKCBRDAPEMJGDDUMXHUMLMMEERBETTXZU'

In [114]:
sa_text == sa_vigener_text

True

#### Is ok decoding?

In [115]:
oa_text = v_c.decode(sa_vigener_text, specified_key)

In [116]:
oa_text

'SLUNECNEATEPLEPOCASICTVRTKEMPROZATIMKONCIVNASLEDUJICICHDVOUDNECHSEZATAHNEVETSINUUZEMIZASAHNEDESTNAHORACHANASEVEROVYCHODEVEVSECHPOLOHACHSNIHOVIKENDUBUDEPOLOJASNOTEPLOTYVNOCIKLESNOUPODNULUPRESDENBUDEMAXIMALNESESTSTUPNU'

In [117]:
oa_text == oa_vigener_text

True

##### Naimplementujte transpoziční šifru:

- a) Columnar Transposition (viz 8. snímek z přednášky)
- b) Row Transposition (viz 9. snímek z přednášky)

Pracujte s přípustnou abecedou obsahující jen znaky anglické abecedy bez mezery. Implementujte jak
šifrování, tak dešifrování. Pro kontrolu použijte klíč K=PROJEV a zašifrujte tento otevřený text: 


In [255]:
class TranspositionCipher:
    def __init__(self, row=True):
        self.row = row

    def generate_key_from_word(self, word):
        chars = list(word)
        chars.sort()
        counter = {}

        for i_c, c in enumerate(chars):
            if c in counter:
                current_value = counter[c]
                new_value = (i_c+1, current_value[1] + 1)
                counter[c] = new_value 
            else:
                counter[c] = (i_c + 1, 1)

        key = ""

        for char in word:
            current_value = counter[char]
            current_index = current_value[0]
            how_many =  current_value[1]
            key += str(current_index - (how_many - 1))
            updated_value = (current_value[0], current_value[1] - 1)
            counter[char] = updated_value 

        return key

    def run(self, text, key):
        size_key = len(key)
        blocks = [text[i:i+size_key] for i in range(0, len(text), size_key)]
        blocks[-1] = blocks[-1] + 'X' * (size_key - len(blocks[-1]))
        indicies = [int(c) for c in key]
        matrix = np.zeros(shape=(len(blocks), size_key), dtype=str)
        for r_i in range(matrix.shape[0]):
            for c_i in range(matrix.shape[1]):
                matrix[r_i, c_i] = blocks[r_i][indicies[c_i]-1] 
        if self.row:
            return "".join(matrix.flatten())
        else:
            return "".join(matrix.flatten('F'))

    def run_with_gen(self, text, word_key):
        number_key = self.generate_key_from_word(word_key)
        return self.run(text, number_key)

    def encode(self, oa_text, word_key):
        return self.run_with_gen(oa_text, word_key)

    def decode(self, sa_text, word_key):
        return self.run_with_gen(oa_text, word_key)

#### Pracujte s přípustnou abecedou obsahující jen znaky anglické abecedy bez mezery. Implementujte jak šifrování, tak dešifrování. Pro kontrolu použijte klíč K=PROJEV a zašifrujte tento otevřený text: 

In [256]:
specified_key = "PROJEV"

In [257]:
oa_trans_text = "RUSKYPREZIDENTVLADIMIRPUTINPRONESLVECTVRTEKTRADICNIPROJEVOSTAVUFEDERACEVENOVALSETOMUZERUSKOMUSIPOKRACOVATVTRANSFORMACIANEMUZESESPOKOJITSESOUCASNYMSTAVEMZMINILSTOUPAJICIPOCETCHUDYCHIZAOSTALOUINFRASTRUKTURUNUTNOUPODPORUDUCHODCUMIMLADYMRODINYMABYSEZVRYTILNEPRIZNIVYDEMOGRAFICKYVYVOJ"

In [258]:
sa_columnar_trans = "YDAPRVTDRSEEAMSICTOAEKESAIOCTCSITROOHIMYEIIDAVKILRPLRAPOFCVOUSAVFIZOSATMTIEYOUSUNPCMYNSTRYRYSZVINSVRIVUAOTRURTSCUPTCSZSJCDAOATTDUUDIYYPVGKJRRNITNCKCJAEESZOOVAMEEJOYEIPPHIAFUNPUDLOAVNNMIVUETMIETTNEVRNEEMKANAMSIUMMLAOUZLRKUODCADBREIOCOPEDUOEEIOTDVLUKPORRNSOSNVNUICHTNRUUROMRMZLZEFY"

In [259]:
sa_columnar_trans_2 = "KILRPLRAPOFCVOUSAVFIZOSATMTIEYOUSUNPCMYNSTRYRYYDAPRVTDRSEEAMSICTOAEKESAIOCTCSITROOHIMYEIIDAVSZVINSVRIVUAOTRURTSCUPTCSZSJCDAOATTDUUDIYYPVGKJUETMIETTNEVRNEEMKANAMSIUMMLAOUZLRKUODCADBREIOCORRNITNCKCJAEESZOOVAMEEJOYEIPPHIAFUNPUDLOAVNNMIVPEDUOEEIOTDVLUKPORRNSOSNVNUICHTNRUUROMRMZLZEFY"

In [260]:
sa_row_trans = "YKSRUPDIZREEALVNTDPRIIMURPNTIOVLSNEETRVCTEDARKTIRPICNOSOVJETEFUAVDECAERVAVOENLMOTSEUSURZEKISUOMPCAROKOTVTVAROFSANRAICMANEZUEMSKOPESOESTJISSACOUNATSYMVIMZEMNOTSILUCIJPAITECPOCCYDHUHSOAIZTIUOALNTSAFRRRUTUKUONTNUUOPDPORHCUUDOIMUDCMMYDLARYNIODMESYABZITYVRLIRPNEZDYVNIEARGMOFVYKICYJVO"

#### Měli byste obdržet šifrový text

###### a)

In [261]:
transposition_cipher = TranspositionCipher(False)

In [262]:
sa_text = transposition_cipher.encode(oa_trans_text, specified_key)

In [263]:
sa_text

'KILRPLRAPOFCVOUSAVFIZOSATMTIEYOUSUNPCMYNSTRYRYXYDAPRVTDRSEEAMSICTOAEKESAIOCTCSITROOHIMYEIIDAVXSZVINSVRIVUAOTRURTSCUPTCSZSJCDAOATTDUUDIYYPVGKJUETMIETTNEVRNEEMKANAMSIUMMLAOUZLRKUODCADBREIOCORRNITNCKCJAEESZOOVAMEEJOYEIPPHIAFUNPUDLOAVNNMIVPEDUOEEIOTDVLUKPORRNSOSNVNUICHTNRUUROMRMZLZEFYX'

In [264]:
sa_text == sa_columnar_trans

False

In [265]:
#Chyba v X ?

sa_text == sa_columnar_trans_2

False

###### b)

In [266]:
transposition_cipher = TranspositionCipher(True)

In [267]:
sa_text = transposition_cipher.encode(oa_trans_text, specified_key)

In [268]:
sa_text == sa_row_trans

False

In [269]:
# Stejny jako priklad druhy